Purpose of this notebook is to experiment with how the explainations are created.
In section 1, we first initialize functions for (loading llm, connecting vector db, and en embeddings model)
In section 2, we experiment and run this flow.

# Initialize

In [ ]:
# install dependencies
!pip install sentence_transformers 
!pip install qdrant_client

In [1]:
# import libraries
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, Batch
import requests
import json
import os

# Change to the parent directory of the notebook
os.chdir('..')

## 

## Load Embeddings model

In [17]:
EMBEDDINGS_MODEL = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', trust_remote_code=True)

## Configure Vector DB

In [2]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://7ef18c4d-2ef6-4fb0-9243-0ac62546593c.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="BR8zsNr5lEYrqJPL4EknUj2oRska2JO1nHwPFawlFMqZIrYMuGZ0Wg",
)

print(qdrant_client.get_collections())

collections=[]


In [25]:
def create_index(index_name):
    qdrant_client.create_collection(
        collection_name=index_name,
        vectors_config=VectorParams(size=len(EMBEDDINGS_MODEL.encode("")), distance=Distance.DOT)
    )

## Configure LLM

In [13]:
API_URL = "https://inf.cl.uni-trier.de/chat/"

def llm(model_name, system_prompt, input_query):
    # Construct the request payload
    payload = {
        "messages": [
            {"content": system_prompt, "role": "system"},
            {"content": input_query, "role": "user"}
        ],
        "model": model_name,
        "options": {}
    }
    
    # Set the request headers
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    
    # Send the POST request
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    
    # Process the response
    if response.status_code == 200:
        print("Response received successfully:")
        response = response.json() # json.dumps(, indent=4)
    else:
        print(f"Failed to retrieve response. Status code: {response.status_code}")

    return response


# Experiments

Here we make experiments with system prompt, 

System prompt contains : what model is supposed to do with the input query. with an exmple output.
Input Query contains, one row of dataset.
explaination variable will have the output of model which will be an explaination.

In [29]:
# Make changes here.
system_prompt = "You are a helpful assistant."
input_query = "Hi"
model_name = "llama3.3:70b-instruct-q6_K"

In [ ]:
response = llm(model_name, system_prompt, input_query)
explaination = response['response']
print(explaination)

### Load Dataset and convert to DF

In [37]:
complete_data = [json.loads(line) for line in open(file_path)]

# Convert the loaded JSONL data to a Pandas DataFrame
complete_df = pd.DataFrame(complete_data)

In [23]:
## incomplete TODO
import pandas as pd
import json

# Load the dataset.json file
file_path = 'Data\\dataset.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Convert the JSON structure into a pandas DataFrame
rows = []
for row in data.items():
    post_tokens = row[1]['post_tokens']
    rationales =  row[1]['rationales']
    # tweet_text = " ".join(post_data["post_tokens"])
    # post_tokens = details.get("post_tokens", [])
    # rationales = details.get("rationales", [])
    
    # Aggregate all rationales
    combined_rationale = [any(rat[i] for rat in rationales) for i in range(len(post_tokens))] if rationales else [0] * len(post_tokens)
    
    # Extract key features
    key_features = [post_tokens[i] for i, is_key in enumerate(combined_rationale) if is_key]
    rows.append({
            "post_id": post_id,
            "tweet_text": tweet_text,
            "key_features": key_features,
            # "annotator_id": annotator["annotator_id"],
            # "label": annotator["label"],
            #"target": ", ".join(annotator["target"]) if annotator["target"] else None
        })
    
    # for annotator in post_data["annotators"]:
    #   rows.append({
    #        "post_id": post_id,
    #        "tweet_text": tweet_text,
    #        "key_features": key_features,
    #        "annotator_id": annotator["annotator_id"],
    #        "label": annotator["label"],
    #        #"target": ", ".join(annotator["target"]) if annotator["target"] else None
    #    })

df = pd.DataFrame(rows)

# Display the first few rows of the dataframe
df.head()

,post_id,tweet_text,key_features
0,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,[]
1,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,[]
2,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,[]
3,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,"[ching, chong]"
4,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,"[bitch, i, hate, white, bitches]"


In [20]:
a[1]

{'post_id': '1179055004553900032_twitter',
 'annotators': [{'label': 'normal', 'annotator_id': 1, 'target': ['None']},
  {'label': 'normal', 'annotator_id': 2, 'target': ['None']},
  {'label': 'normal', 'annotator_id': 3, 'target': ['None']}],
 'rationales': [],
 'post_tokens': ['i',
  'dont',
  'think',
  'im',
  'getting',
  'my',
  'baby',
  'them',
  'white',
  '9',
  'he',
  'has',
  'two',
  'white',
  'j',
  'and',
  'nikes',
  'not',
  'even',
  'touched']}

In [6]:
for a in data.items():
    print(a)
    break

('1179055004553900032_twitter', {'post_id': '1179055004553900032_twitter', 'annotators': [{'label': 'normal', 'annotator_id': 1, 'target': ['None']}, {'label': 'normal', 'annotator_id': 2, 'target': ['None']}, {'label': 'normal', 'annotator_id': 3, 'target': ['None']}], 'rationales': [], 'post_tokens': ['i', 'dont', 'think', 'im', 'getting', 'my', 'baby', 'them', 'white', '9', 'he', 'has', 'two', 'white', 'j', 'and', 'nikes', 'not', 'even', 'touched']})


# Deploy

In [26]:
index_name = "test_index"
create_index(index_name)

In [27]:
print(qdrant_client.get_collections())

collections=[CollectionDescription(name='test_index')]
